### Besoin du client

Nous souhaitons acquérir un outil d’analyse de nos ventes pour déterminer et évaluer nos ventes de
produits à partir des données de ventes exportées par notre caisse enregistreuse.

Pour se faire nous désirons connaître:

● Nos ventes de produits par catégorie (pains, viennoiseries, pâtisseries et autres), en terme quantitative et financière


● Quelles sont nos meilleures ventes par mois ? En opposition y a t il des produits qui sont peu vendus ? idem avec nos catégories de produits


● Le panier moyen de nos ventes par mois et par semaine

### Bibliothèque

In [ ]:
import numpy as np
import pandas as pd
import re

### Exploration des données

In [ ]:
boulangerie = pd.read_csv("https://drive.google.com/uc?export=view&id=19_KerlB0JDgceVKRVPmpFMrj4XSJQqnU")
display(boulangerie.head())

,Unnamed: 0,date,time,ticket_number,article,Quantity,unit_price
0,0,2021-01-02,08:38,150040.0,BAGUETTE,1.0,"0,90 €"
1,1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3.0,"1,20 €"
2,4,2021-01-02,09:14,150041.0,PAIN AU CHOCOLAT,2.0,"1,20 €"
3,5,2021-01-02,09:14,150041.0,PAIN,1.0,"1,15 €"
4,8,2021-01-02,09:25,150042.0,TRADITIONAL BAGUETTE,5.0,"1,20 €"


La colonne "Unnamed: 0" semble être un doublon de l'index. On remarque qu'on passe de 1 à 4 puis de 5 à 8, on peut supposer que des lignes contenant des valeurs NaN ont été supprimées et que le réindexage des lignes n'a pas été fait, supprimer la colonne n'aura pas d'impact sur la suite de l'analyse.

In [ ]:
# Suppression de la colonne "Unnamed: 0" car c'est un doublon de l'index
boulangerie = boulangerie.drop(columns='Unnamed: 0')
display(boulangerie.head())

,date,time,ticket_number,article,Quantity,unit_price
0,2021-01-02,08:38,150040.0,BAGUETTE,1.0,"0,90 €"
1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3.0,"1,20 €"
2,2021-01-02,09:14,150041.0,PAIN AU CHOCOLAT,2.0,"1,20 €"
3,2021-01-02,09:14,150041.0,PAIN,1.0,"1,15 €"
4,2021-01-02,09:25,150042.0,TRADITIONAL BAGUETTE,5.0,"1,20 €"


In [ ]:
# Vérification des données, du type des colonnes
print(boulangerie.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234005 entries, 0 to 234004
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   date           234005 non-null  object 
 1   time           234005 non-null  object 
 2   ticket_number  234005 non-null  float64
 3   article        234005 non-null  object 
 4   Quantity       234005 non-null  float64
 5   unit_price     234005 non-null  object 
dtypes: float64(2), object(4)
memory usage: 10.7+ MB
None


D'après les vérifications, il semble que les colonnes "date" et "time" sont au format "object", nous devrions les convertir au format datetime. Si la colonne "ticket_number" ne contient pas de float nous pourrions la convertir en int et enfin la colonne "unit_price" semble être au format "object", on devrait peut être utilisé regex pour ne conserver que le prix sans la devise étant donné que tous les prix sont en euros. Il semblerait également qu'il n'y a pas de valeurs manquantes

In [ ]:
# Vérification des données avec describe
display(boulangerie.describe(include='all'))

,date,time,ticket_number,article,Quantity,unit_price
count,234005,234005,234005.000000,234005,234005.000000,234005
unique,600,683,NaN,149,NaN,123
top,2022-08-14,11:43,NaN,TRADITIONAL BAGUETTE,NaN,"1,20 €"
freq,997,859,NaN,67689,NaN,49080
mean,NaN,NaN,219201.258738,NaN,1.538377,NaN
std,NaN,NaN,40053.223896,NaN,1.289603,NaN
min,NaN,NaN,150040.000000,NaN,-200.000000,NaN
25%,NaN,NaN,184754.000000,NaN,1.000000,NaN
50%,NaN,NaN,218807.000000,NaN,1.000000,NaN
75%,NaN,NaN,253927.000000,NaN,2.000000,NaN


Finalement, est-ce utile de garder la colonne "time" ? Peut être pour analyser les heures de rush ? (à voir)

In [ ]:
# Convertir la colonne "date" au format date
boulangerie['date'] = pd.to_datetime(boulangerie['date'])
print(boulangerie.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234005 entries, 0 to 234004
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   date           234005 non-null  datetime64[ns]
 1   time           234005 non-null  object        
 2   ticket_number  234005 non-null  float64       
 3   article        234005 non-null  object        
 4   Quantity       234005 non-null  float64       
 5   unit_price     234005 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 10.7+ MB
None


In [ ]:
boulangerie['time'] = boulangerie['time'].apply(lambda x: x + ':00' if len(x.split(':')) == 2 else x)

boulangerie['time'] = pd.to_datetime(boulangerie['time'], format='%H:%M:%S').dt.time

In [ ]:
# Vérifier si des valeurs float existent dans 'ticket_number'
if any(isinstance(x, float) for x in boulangerie['ticket_number'].unique()):
    print("Il y a des valeurs float dans 'ticket_number'.")
else:
    print("Il n'y a pas de valeurs float dans 'ticket_number'.")

Il y a des valeurs float dans 'ticket_number'.


Il semblerait qu'il existe des valeurs float dans la colonne 'ticket_number', nous ne pouvons pas la convertir en int.

In [ ]:
# Vérification si tous les prix sont en euros
if all(boulangerie['unit_price'].str.contains('€')):
    print("Tous les prix sont en euros.")

Tous les prix sont en euros.


In [ ]:
# Fonction pour extraire le prix avec regex et convertir en float
def extract_price(price_str):
    # Utilisation de regex pour extraire le nombre
    match = re.search(r'(\d+,\d+)', price_str)
    if match:
        # Remplacer la virgule par un point et convertir en float
        return float(match.group(1).replace(',', '.'))
    return None

# Appliquer la fonction à la colonne 'unit_price'
boulangerie['unit_price'] = boulangerie['unit_price'].apply(extract_price)

display(boulangerie.head())

,date,time,ticket_number,article,Quantity,unit_price
0,2021-01-02,08:38:00,150040.0,BAGUETTE,1.0,0.90
1,2021-01-02,08:38:00,150040.0,PAIN AU CHOCOLAT,3.0,1.20
2,2021-01-02,09:14:00,150041.0,PAIN AU CHOCOLAT,2.0,1.20
3,2021-01-02,09:14:00,150041.0,PAIN,1.0,1.15
4,2021-01-02,09:25:00,150042.0,TRADITIONAL BAGUETTE,5.0,1.20


In [ ]:
# Vérifier à nouveau les données
print(boulangerie.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234005 entries, 0 to 234004
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   date           234005 non-null  datetime64[ns]
 1   time           234005 non-null  object        
 2   ticket_number  234005 non-null  float64       
 3   article        234005 non-null  object        
 4   Quantity       234005 non-null  float64       
 5   unit_price     234005 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 10.7+ MB
None


In [ ]:
# D'après le describe, il existe 149 valeurs uniques dans la colonne "article"
print(boulangerie['article'].unique())

['BAGUETTE' 'PAIN AU CHOCOLAT' 'PAIN' 'TRADITIONAL BAGUETTE' 'CROISSANT'
 'BANETTE' 'BANETTINE' 'SPECIAL BREAD' 'COUPE' 'SAND JB EMMENTAL'
 'KOUIGN AMANN' 'BOULE 200G' 'BOULE 400G' 'GAL FRANGIPANE 6P' 'CAMPAGNE'
 'MOISSON' 'CAFE OU EAU' 'BRIOCHE' 'CEREAL BAGUETTE' 'SEIGLE' 'COMPLET'
 'DIVERS PATISSERIE' 'GAL FRANGIPANE 4P' 'COOKIE' 'FICELLE'
 'PAIN AUX RAISINS' 'GAL POMME 6P' 'GAL POMME 4P' 'FINANCIER X5'
 'VIK BREAD' 'DIVERS VIENNOISERIE' 'GACHE' 'SANDWICH COMPLET'
 'PAIN BANETTE' 'GRAND FAR BRETON' 'QUIM BREAD' 'SPECIAL BREAD KG'
 'GD KOUIGN AMANN' 'BOULE POLKA' 'DEMI BAGUETTE' 'CHAUSSON AUX POMMES'
 'BAGUETTE GRAINE' 'DIVERS CONFISERIE' 'SUCETTE' 'DIVERS BOULANGERIE'
 'BOISSON 33CL' 'PATES' 'FORMULE SANDWICH' 'DIVERS SANDWICHS'
 'CROISSANT AMANDES' 'PAIN CHOCO AMANDES' 'SACHET VIENNOISERIE' 'NANTAIS'
 'CHOCOLAT' 'PAIN S/SEL' 'FONDANT CHOCOLAT' 'GAL POIRE CHOCO 6P'
 'GAL POIRE CHOCO 4P' 'GALETTE 8 PERS' 'SAND JB' 'SACHET DE CROUTON'
 'GRANDE SUCETTE' 'DEMI PAIN' 'TARTELETTE' 'FLAN' '

In [ ]:
# Compter le nombre d'occurrences de chaque valeur unique dans la colonne 'article'
total_articles = boulangerie['article'].value_counts()
display(total_articles)

,count
article,
TRADITIONAL BAGUETTE,67689
COUPE,20470
BAGUETTE,15292
BANETTE,15130
CROISSANT,11508
...,...
PAIN NOIR,1
BUCHE 8PERS,1
CRUMBLECARAMEL OU PISTAE,1


Faudrait-il utiliser du NLP pour regrouper des mots ensemble ou cela risquerait-il de fausser les données ? Par exemple, "BAGUETTE" et "BANETTE" se rapprochent, pareil pour ces catégories : 'PLAT 6.50E' 'PLAT 7.60E' 'PLAT 7.00' 'PLAT' 'PLAT 8.30E'. (à voir)

In [ ]:
# Vérifier l'article "PLAT 6.50E"
mask = boulangerie['article'] == 'PLAT 6.50E'
boulangerie.loc[mask]

,date,time,ticket_number,article,Quantity,unit_price
145203,2022-02-28,07:45:00,235581.0,PLAT 6.50E,1.0,6.5
145204,2022-02-28,07:59:00,235582.0,PLAT 6.50E,-1.0,6.5


Ceci s'apparente à un retour de l'article "PLAT 6.50E". Devrions nous prendre en compte un éventuel remboursement ?

In [ ]:
# Vérifier l'article "PLAT 7.60E"
mask_2 = boulangerie['article'] == 'PLAT 7.60E'
boulangerie.loc[mask_2]

,date,time,ticket_number,article,Quantity,unit_price
145292,2022-02-28,10:56:00,235640.0,PLAT 7.60E,1.0,7.6
145342,2022-02-28,12:02:00,235675.0,PLAT 7.60E,1.0,7.6
145854,2022-03-02,07:40:00,236002.0,PLAT 7.60E,1.0,7.6
145868,2022-03-02,07:55:00,236009.0,PLAT 7.60E,1.0,7.6
146176,2022-03-03,09:42:00,236194.0,PLAT 7.60E,2.0,7.6
...,...,...,...,...,...,...
233574,2022-09-29,12:16:00,288653.0,PLAT 7.60E,2.0,7.6
233796,2022-09-30,10:50:00,288790.0,PLAT 7.60E,1.0,7.6
233817,2022-09-30,11:09:00,288801.0,PLAT 7.60E,1.0,7.6
233889,2022-09-30,12:19:00,288845.0,PLAT 7.60E,1.0,7.6


In [ ]:
# Vérifier l'article "PLAT 7.00"
mask_3 = boulangerie['article'] == 'PLAT 7.00'
boulangerie.loc[mask_3]

,date,time,ticket_number,article,Quantity,unit_price
145698,2022-03-01,12:00:00,235900.0,PLAT 7.00,1.0,7.0
145705,2022-03-01,12:07:00,235904.0,PLAT 7.00,2.0,7.0
145746,2022-03-01,12:41:00,235931.0,PLAT 7.00,-1.0,7.0
151947,2022-03-25,10:05:00,239759.0,PLAT 7.00,1.0,7.0
152018,2022-03-25,11:58:00,239808.0,PLAT 7.00,1.0,7.0
...,...,...,...,...,...,...
229640,2022-09-16,12:46:00,286230.0,PLAT 7.00,1.0,7.0
230738,2022-09-20,11:15:00,286846.0,PLAT 7.00,1.0,7.0
230804,2022-09-20,12:25:00,286892.0,PLAT 7.00,1.0,7.0
231005,2022-09-21,11:36:00,287026.0,PLAT 7.00,1.0,7.0


In [ ]:
# Vérifier l'article "PLAT"
mask_4 = boulangerie['article'] == 'PLAT'
boulangerie.loc[mask_4]

,date,time,ticket_number,article,Quantity,unit_price
146308,2022-03-03,12:36:00,236289.0,PLAT,1.0,4.9
154783,2022-04-04,12:46:00,241459.0,PLAT,1.0,1.0
154994,2022-04-05,12:00:00,241592.0,PLAT,1.0,3.0
159809,2022-04-19,13:11:00,244521.0,PLAT,1.0,14.0
186701,2022-07-01,13:02:00,260940.0,PLAT,1.0,9.1
189733,2022-07-08,12:22:00,262813.0,PLAT,1.0,4.9
220432,2022-08-22,13:06:00,280573.0,PLAT,1.0,1.5
220994,2022-08-23,17:14:00,280933.0,PLAT,2.0,1.5


In [ ]:
# Vérifier l'article "PLAT 8.30E"
mask_5 = boulangerie['article'] == 'PLAT 8.30E'
boulangerie.loc[mask_5]

,date,time,ticket_number,article,Quantity,unit_price
147524,2022-03-06,12:00:00,236990.0,PLAT 8.30E,1.0,8.3
152906,2022-03-28,09:15:00,240306.0,PLAT 8.30E,1.0,8.3
152968,2022-03-28,11:03:00,240348.0,PLAT 8.30E,1.0,8.3
153570,2022-03-31,09:16:00,240745.0,PLAT 8.30E,1.0,8.3
154659,2022-04-04,09:49:00,241374.0,PLAT 8.30E,1.0,8.3
...,...,...,...,...,...,...
232529,2022-09-26,09:59:00,287945.0,PLAT 8.30E,1.0,8.3
233079,2022-09-27,18:42:00,288322.0,PLAT 8.30E,1.0,8.3
233556,2022-09-29,11:55:00,288640.0,PLAT 8.30E,1.0,8.3
233569,2022-09-29,12:13:00,288651.0,PLAT 8.30E,1.0,8.3


Etant donné que l'article 'PLAT' semble avoir des prix différents, nous devrions peut être bel et bien envisager d'utiliser le NLP ?

In [ ]:
# Vérifier l'article "."
mask_dot = boulangerie['article'] == '.'
boulangerie.loc[mask_dot]

,date,time,ticket_number,article,Quantity,unit_price
15360,2021-03-04,12:32:00,159219.0,.,2.0,0.0
19904,2021-03-18,12:59:00,161853.0,.,1.0,0.0
24959,2021-04-04,09:53:00,164878.0,.,1.0,0.0
33569,2021-04-27,16:48:00,170079.0,.,1.0,0.0
61792,2021-07-10,13:25:00,186662.0,.,2.0,0.0


In [ ]:
# Vérifier l'article "ARTICLE 295"
mask_295 = boulangerie['article'] == 'ARTICLE 295'
boulangerie.loc[mask_295]

,date,time,ticket_number,article,Quantity,unit_price
118498,2021-11-08,20:00:00,219725.0,ARTICLE 295,1.0,0.0


### KPI : Nos ventes de produits par catégorie (pains, viennoiseries, pâtisseries et autres), en terme quantitative et financière

In [ ]:
# Dictionnaire des catégories avec les articles correspondants
categories = {
    'Pains': [
        'BAGUETTE', 'TRADITIONAL BAGUETTE', 'BANETTE', 'BANETTINE', 'SPECIAL BREAD', 'COUPE',
        'BOULE 200G', 'BOULE 400G', 'CAMPAGNE', 'MOISSON', 'CEREAL BAGUETTE', 'SEIGLE', 'COMPLET',
        'PAIN', 'FICELLE', 'PAIN BANETTE', 'QUIM BREAD', 'SPECIAL BREAD KG', 'BOULE POLKA',
        'DEMI BAGUETTE', 'BAGUETTE GRAINE', 'PAIN S/SEL', 'DEMI PAIN', 'PAIN DE MIE', 'PAIN GRAINES',
        'PAIN NOIR', 'GUERANDAIS', 'BAGUETTE APERO'
    ],
    'Viennoiseries': [
        'PAIN AU CHOCOLAT', 'CROISSANT', 'BRIOCHE', 'PAIN AUX RAISINS', 'GACHE', 'SACHET VIENNOISERIE',
        'CROISSANT AMANDES', 'PAIN CHOCO AMANDES', 'VIENNOISE', 'SACHET DE VIENNOISERIE', 'PAIN SUISSE PEPITO'
    ],
    'Pâtisserie': [
        'KOUIGN AMANN', 'GAL FRANGIPANE 6P', 'DIVERS PATISSERIE', 'GAL FRANGIPANE 4P', 'COOKIE',
        'GAL POMME 6P', 'GAL POMME 4P', 'FINANCIER X5', 'GRAND FAR BRETON', 'GD KOUIGN AMANN',
        'CHAUSSON AUX POMMES', 'NANTAIS', 'FONDANT CHOCOLAT', 'GAL POIRE CHOCO 6P', 'GAL POIRE CHOCO 4P',
        'GALETTE 8 PERS', 'TARTELETTE', 'FLAN', 'PARIS BREST', 'SAVARIN', 'FLAN ABRICOT',
        'MILLES FEUILLES', 'CHOU CHANTILLY', 'ECLAIR', 'ROYAL 4P', 'TARTE FRUITS 6P',
        'TARTE FRUITS 4P', 'NOIX JAPONAISE', 'ROYAL 6P', 'ECLAIR FRAISE PISTACHE', 'GD FAR BRETON',
        'TRIANGLES', 'TROPEZIENNE', 'TROPEZIENNE FRAMBOISE', 'ROYAL', 'TARTE FRAISE 6P',
        'TARTELETTE FRAISE', 'TARTE FRAISE 4PER', 'FRAISIER', 'NID DE POULE', 'TARTELETTE CHOC',
        'CRUMBLE', 'FINANCIER', 'ST HONORE', 'BROWNIES', 'RELIGIEUSE', 'DELICETROPICAL',
        'CRUMBLECARAMEL OU PISTAE', 'PT NANTAIS', 'GD NANTAIS', 'DOUCEUR D HIVER', 'TROIS CHOCOLAT',
        'TARTE FINE', 'ENTREMETS', 'BRIOCHE DE NOEL', 'FRAMBOISIER', 'BUCHE 4PERS',
        'BUCHE 6PERS', 'BUCHE 8PERS', 'REDUCTION SUCREES 12', 'REDUCTION SUCREES 24', 'BOTTEREAU',
        'MERINGUE', 'PALMIER', 'PAILLE', 'CARAMEL NOIX', 'MACARON', '12 MACARON', 'ARMORICAIN',
        'PLAQUE TARTE 25P', 'SABLE F  P', 'TULIPE', 'TARTELETTE COCKTAIL'
    ],
    'Plats préparés': [
        'SAND JB EMMENTAL', 'SANDWICH COMPLET', 'DIVERS SANDWICHS',
        'PLATPREPARE6,50', 'PLATPREPARE5,50', 'PLATPREPARE7,00',
        'PLATPREPARE6,00', 'PLAT 6.50E', 'PLAT 7.60E', 'PLAT 7.00', 'PLAT', 'PLAT 8.30E'
    ],
    'Formules': ['FORMULE SANDWICH', 'FORMULE PLAT PREPARE'
    ],
    'Boissons': ['CAFE OU EAU', 'BOISSON 33CL', 'DIVERS BOISSONS', 'THE'],
    'Confiserie': ['DIVERS CONFISERIE', 'SUCETTE', 'SACHET DE CROUTON', 'GRANDE SUCETTE', 'CHOCOLAT'],
    'Divers': ['DIVERS BOULANGERIE', 'ARTICLE 295', '.']
}

# Fonction pour assigner une catégorie
def assign_category(article):
    for category, articles in categories.items():
        if article in articles:
            return category
    return 'Divers'  # Si aucun article ne correspond

# Ajouter la colonne catégorie
boulangerie['category'] = boulangerie['article'].apply(assign_category)

# Afficher le DataFrame avec la nouvelle colonne
display(boulangerie)

,date,time,ticket_number,article,Quantity,unit_price,category
0,2021-01-02,08:38:00,150040.0,BAGUETTE,1.0,0.90,Pains
1,2021-01-02,08:38:00,150040.0,PAIN AU CHOCOLAT,3.0,1.20,Viennoiseries
2,2021-01-02,09:14:00,150041.0,PAIN AU CHOCOLAT,2.0,1.20,Viennoiseries
3,2021-01-02,09:14:00,150041.0,PAIN,1.0,1.15,Pains
4,2021-01-02,09:25:00,150042.0,TRADITIONAL BAGUETTE,5.0,1.20,Pains
...,...,...,...,...,...,...,...
234000,2022-09-30,18:52:00,288911.0,COUPE,1.0,0.15,Pains
234001,2022-09-30,18:52:00,288911.0,BOULE 200G,1.0,1.20,Pains
234002,2022-09-30,18:52:00,288911.0,COUPE,2.0,0.15,Pains
234003,2022-09-30,18:55:00,288912.0,TRADITIONAL BAGUETTE,1.0,1.30,Pains


In [ ]:
# Vérifier les données
boulangerie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234005 entries, 0 to 234004
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   date           234005 non-null  datetime64[ns]
 1   time           234005 non-null  object        
 2   ticket_number  234005 non-null  float64       
 3   article        234005 non-null  object        
 4   Quantity       234005 non-null  float64       
 5   unit_price     234005 non-null  float64       
 6   category       234005 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 12.5+ MB


**MCA : Après le nettoyage des données, faudrait-il rajouter une colonne TOTAL par article (Quantity x unit-price) afin de connaitre le CA réalisé pour chaque article pour chaque année (en 2021 et en 2022) **

In [ ]:
# Création de la colonne CA
boulangerie['CA'] = (boulangerie['Quantity']*boulangerie['unit_price']).round(2)

display(boulangerie)

,date,time,ticket_number,article,Quantity,unit_price,category,CA
0,2021-01-02,08:38:00,150040.0,BAGUETTE,1.0,0.90,Pains,0.90
1,2021-01-02,08:38:00,150040.0,PAIN AU CHOCOLAT,3.0,1.20,Viennoiseries,3.60
2,2021-01-02,09:14:00,150041.0,PAIN AU CHOCOLAT,2.0,1.20,Viennoiseries,2.40
3,2021-01-02,09:14:00,150041.0,PAIN,1.0,1.15,Pains,1.15
4,2021-01-02,09:25:00,150042.0,TRADITIONAL BAGUETTE,5.0,1.20,Pains,6.00
...,...,...,...,...,...,...,...,...
234000,2022-09-30,18:52:00,288911.0,COUPE,1.0,0.15,Pains,0.15
234001,2022-09-30,18:52:00,288911.0,BOULE 200G,1.0,1.20,Pains,1.20
234002,2022-09-30,18:52:00,288911.0,COUPE,2.0,0.15,Pains,0.30
234003,2022-09-30,18:55:00,288912.0,TRADITIONAL BAGUETTE,1.0,1.30,Pains,1.30


In [ ]:
# Création de la colonne 'year'
boulangerie['year'] = boulangerie['date'].dt.year

display(boulangerie.head())

,date,time,ticket_number,article,Quantity,unit_price,category,CA,year
0,2021-01-02,08:38:00,150040.0,BAGUETTE,1.0,0.90,Pains,0.90,2021
1,2021-01-02,08:38:00,150040.0,PAIN AU CHOCOLAT,3.0,1.20,Viennoiseries,3.60,2021
2,2021-01-02,09:14:00,150041.0,PAIN AU CHOCOLAT,2.0,1.20,Viennoiseries,2.40,2021
3,2021-01-02,09:14:00,150041.0,PAIN,1.0,1.15,Pains,1.15,2021
4,2021-01-02,09:25:00,150042.0,TRADITIONAL BAGUETTE,5.0,1.20,Pains,6.00,2021


In [ ]:
# Création de la colonne 'month'
boulangerie['month'] = boulangerie['date'].dt.month

display(boulangerie.head())

,date,time,ticket_number,article,Quantity,unit_price,category,CA,year,month
0,2021-01-02,08:38:00,150040.0,BAGUETTE,1.0,0.90,Pains,0.90,2021,1
1,2021-01-02,08:38:00,150040.0,PAIN AU CHOCOLAT,3.0,1.20,Viennoiseries,3.60,2021,1
2,2021-01-02,09:14:00,150041.0,PAIN AU CHOCOLAT,2.0,1.20,Viennoiseries,2.40,2021,1
3,2021-01-02,09:14:00,150041.0,PAIN,1.0,1.15,Pains,1.15,2021,1
4,2021-01-02,09:25:00,150042.0,TRADITIONAL BAGUETTE,5.0,1.20,Pains,6.00,2021,1


In [ ]:
# Création de la colonne 'week'
boulangerie['week'] = boulangerie['date'].dt.isocalendar().week

display(boulangerie.tail())

,date,time,ticket_number,article,Quantity,unit_price,category,CA,year,month,week
234000,2022-09-30,18:52:00,288911.0,COUPE,1.0,0.15,Pains,0.15,2022,9,39
234001,2022-09-30,18:52:00,288911.0,BOULE 200G,1.0,1.20,Pains,1.20,2022,9,39
234002,2022-09-30,18:52:00,288911.0,COUPE,2.0,0.15,Pains,0.30,2022,9,39
234003,2022-09-30,18:55:00,288912.0,TRADITIONAL BAGUETTE,1.0,1.30,Pains,1.30,2022,9,39
234004,2022-09-30,18:56:00,288913.0,TRADITIONAL BAGUETTE,1.0,1.30,Pains,1.30,2022,9,39


In [ ]:
boulangerie['week'].value_counts()

,count
week,
32,10523
33,9797
31,9450
28,9289
30,8845
29,8607
34,8026
27,6702
21,6579


In [ ]:
# Vérifier les semaines
mask_week = boulangerie['week'] == 53
boulangerie.loc[mask_week]

,date,time,ticket_number,article,Quantity,unit_price,category,CA,year,month,week
0,2021-01-02,08:38:00,150040.0,BAGUETTE,1.0,0.90,Pains,0.90,2021,1,53
1,2021-01-02,08:38:00,150040.0,PAIN AU CHOCOLAT,3.0,1.20,Viennoiseries,3.60,2021,1,53
2,2021-01-02,09:14:00,150041.0,PAIN AU CHOCOLAT,2.0,1.20,Viennoiseries,2.40,2021,1,53
3,2021-01-02,09:14:00,150041.0,PAIN,1.0,1.15,Pains,1.15,2021,1,53
4,2021-01-02,09:25:00,150042.0,TRADITIONAL BAGUETTE,5.0,1.20,Pains,6.00,2021,1,53
...,...,...,...,...,...,...,...,...,...,...,...
735,2021-01-03,13:50:00,150443.0,COUPE,1.0,0.15,Pains,0.15,2021,1,53
736,2021-01-03,13:50:00,150443.0,MOISSON,1.0,2.00,Pains,2.00,2021,1,53
737,2021-01-03,13:51:00,150444.0,CEREAL BAGUETTE,2.0,1.25,Pains,2.50,2021,1,53
738,2021-01-03,13:55:00,150445.0,QUIM BREAD,1.0,1.00,Pains,1.00,2021,1,53


La semaine 53 débute le lundi 28 décembre 2020 et se termine le 3 janvier 2021, donc dans le dataframe le 02/01/2021 et le 03/01/2021 font partis de la semaine 53.

In [ ]:
# Réajuster l'emplacement des colonnes
boulangerie = boulangerie[['date', 'year', 'month', 'week', 'time', 'ticket_number', 'article', 'category', 'Quantity', 'unit_price', 'CA']]

display(boulangerie)

,date,year,month,week,time,ticket_number,article,category,Quantity,unit_price,CA
0,2021-01-02,2021,1,53,08:38:00,150040.0,BAGUETTE,Pains,1.0,0.90,0.90
1,2021-01-02,2021,1,53,08:38:00,150040.0,PAIN AU CHOCOLAT,Viennoiseries,3.0,1.20,3.60
2,2021-01-02,2021,1,53,09:14:00,150041.0,PAIN AU CHOCOLAT,Viennoiseries,2.0,1.20,2.40
3,2021-01-02,2021,1,53,09:14:00,150041.0,PAIN,Pains,1.0,1.15,1.15
4,2021-01-02,2021,1,53,09:25:00,150042.0,TRADITIONAL BAGUETTE,Pains,5.0,1.20,6.00
...,...,...,...,...,...,...,...,...,...,...,...
234000,2022-09-30,2022,9,39,18:52:00,288911.0,COUPE,Pains,1.0,0.15,0.15
234001,2022-09-30,2022,9,39,18:52:00,288911.0,BOULE 200G,Pains,1.0,1.20,1.20
234002,2022-09-30,2022,9,39,18:52:00,288911.0,COUPE,Pains,2.0,0.15,0.30
234003,2022-09-30,2022,9,39,18:55:00,288912.0,TRADITIONAL BAGUETTE,Pains,1.0,1.30,1.30


In [ ]:
# Revérifier les données
boulangerie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234005 entries, 0 to 234004
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   date           234005 non-null  datetime64[ns]
 1   year           234005 non-null  int32         
 2   month          234005 non-null  int32         
 3   week           234005 non-null  UInt32        
 4   time           234005 non-null  object        
 5   ticket_number  234005 non-null  float64       
 6   article        234005 non-null  object        
 7   category       234005 non-null  object        
 8   Quantity       234005 non-null  float64       
 9   unit_price     234005 non-null  float64       
 10  CA             234005 non-null  float64       
dtypes: UInt32(1), datetime64[ns](1), float64(4), int32(2), object(3)
memory usage: 17.2+ MB


In [ ]:
# Calculer le total des ventes
somme_CA = boulangerie['CA'].sum().round(2)

print(somme_CA)

558640.85


### Le panier moyen de nos ventes par mois et par semaine

In [ ]:
# Grouper par 'ticket_number' et sommer les valeurs de la colonne 'CA'
panier_total = boulangerie.groupby(by='ticket_number', dropna=False)['CA'].sum().round(2)

# Afficher le résultat
display(panier_total)

,CA
ticket_number,
150040.0,4.50
150041.0,3.55
150042.0,6.00
150043.0,5.10
150044.0,1.05
...,...
288908.0,2.70
288910.0,1.30
288911.0,12.15


In [ ]:
panier_total.info()

<class 'pandas.core.series.Series'>
Index: 136451 entries, 150040.0 to 288913.0
Series name: CA
Non-Null Count   Dtype  
--------------   -----  
136451 non-null  float64
dtypes: float64(1)
memory usage: 2.1 MB


In [ ]:
# Vérifier la somme pour le premier ticket
mask_panier = boulangerie['ticket_number'] == 150040.0
boulangerie.loc[mask_panier]

,date,year,month,week,time,ticket_number,article,category,Quantity,unit_price,CA
0,2021-01-02,2021,1,53,08:38:00,150040.0,BAGUETTE,Pains,1.0,0.9,0.9
1,2021-01-02,2021,1,53,08:38:00,150040.0,PAIN AU CHOCOLAT,Viennoiseries,3.0,1.2,3.6


In [ ]:
panier_total.describe(include='all')

,CA
count,136451.000000
mean,4.094077
std,4.536965
min,-200.000000
25%,1.300000
50%,2.500000
75%,5.100000
max,200.000000


In [ ]:
panier_ticket = boulangerie.merge(panier_total, how='left', on='ticket_number')

In [ ]:
panier_ticket.head()

,date,year,month,week,time,ticket_number,article,category,Quantity,unit_price,CA_x,CA_y
0,2021-01-02,2021,1,53,08:38:00,150040.0,BAGUETTE,Pains,1.0,0.90,0.90,4.50
1,2021-01-02,2021,1,53,08:38:00,150040.0,PAIN AU CHOCOLAT,Viennoiseries,3.0,1.20,3.60,4.50
2,2021-01-02,2021,1,53,09:14:00,150041.0,PAIN AU CHOCOLAT,Viennoiseries,2.0,1.20,2.40,3.55
3,2021-01-02,2021,1,53,09:14:00,150041.0,PAIN,Pains,1.0,1.15,1.15,3.55
4,2021-01-02,2021,1,53,09:25:00,150042.0,TRADITIONAL BAGUETTE,Pains,5.0,1.20,6.00,6.00


In [ ]:
panier_ticket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234005 entries, 0 to 234004
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   date           234005 non-null  datetime64[ns]
 1   year           234005 non-null  int32         
 2   month          234005 non-null  int32         
 3   week           234005 non-null  UInt32        
 4   time           234005 non-null  object        
 5   ticket_number  234005 non-null  float64       
 6   article        234005 non-null  object        
 7   category       234005 non-null  object        
 8   Quantity       234005 non-null  float64       
 9   unit_price     234005 non-null  float64       
 10  CA_x           234005 non-null  float64       
 11  CA_y           234005 non-null  float64       
dtypes: UInt32(1), datetime64[ns](1), float64(5), int32(2), object(3)
memory usage: 19.0+ MB


In [ ]:
# Supprimer les doublons
panier_ticket = panier_ticket.drop_duplicates(subset='ticket_number')

# Supprimer les colonnes dont on a pas besoin
panier_ticket = panier_ticket.drop(columns=['article', 'category', 'Quantity', 'unit_price', 'CA_x'])

# Renommer la colonne 'CA_y'
panier_ticket = panier_ticket.rename(columns={'CA_y' : 'total_ticket'})

display(panier_ticket)

,date,year,month,week,time,ticket_number,total_ticket
0,2021-01-02,2021,1,53,08:38:00,150040.0,4.50
2,2021-01-02,2021,1,53,09:14:00,150041.0,3.55
4,2021-01-02,2021,1,53,09:25:00,150042.0,6.00
5,2021-01-02,2021,1,53,09:25:00,150043.0,5.10
7,2021-01-02,2021,1,53,09:27:00,150044.0,1.05
...,...,...,...,...,...,...,...
233996,2022-09-30,2022,9,39,18:34:00,288908.0,2.70
233997,2022-09-30,2022,9,39,18:39:00,288910.0,1.30
233998,2022-09-30,2022,9,39,18:52:00,288911.0,12.15
234003,2022-09-30,2022,9,39,18:55:00,288912.0,1.30


In [ ]:
# Réindexage des lignes après avoir supprimer les doublons
panier_ticket = panier_ticket.reset_index(drop=True)

display(panier_ticket.head())

,date,year,month,week,time,ticket_number,total_ticket
0,2021-01-02,2021,1,53,08:38:00,150040.0,4.50
1,2021-01-02,2021,1,53,09:14:00,150041.0,3.55
2,2021-01-02,2021,1,53,09:25:00,150042.0,6.00
3,2021-01-02,2021,1,53,09:25:00,150043.0,5.10
4,2021-01-02,2021,1,53,09:27:00,150044.0,1.05


In [ ]:
# Calculer le total des ventes
somme_vente = panier_ticket['total_ticket'].sum()

print(somme_vente)

558640.85


In [ ]:
# Calculer le nombre total de ticket
total_ticket = panier_ticket['ticket_number'].value_counts().sum()

print(total_ticket)

136451


In [ ]:
# Calculer panier moyen général
panier_moyen = (somme_vente / total_ticket).round(2)

print(panier_moyen)

4.09


In [ ]:
# Filtrer le DataFrame directement
mask_2021 = (panier_ticket['month'] == 1) & (panier_ticket['year'] == 2021)

# Calculer la somme de la colonne 'total_ticket' pour les lignes filtrées
somme_mask = panier_ticket.loc[mask_2021, 'total_ticket'].sum().round(2)
panier_moyen = panier_ticket.loc[mask_2021, 'total_ticket'].mean().round(2)

print(somme_mask)
print(panier_moyen)

15258.67
3.89


In [ ]:
# Calcul du panier moyen par semaine
panier_moyen_semaine = panier_ticket.groupby(['year', 'week'])['total_ticket'].mean().round(2).reset_index()
panier_moyen_semaine = panier_moyen_semaine.rename(columns={'total_ticket': 'panier_moyen_semaine'})

# Calcul du panier moyen par mois
panier_moyen_mois = panier_ticket.groupby(['year', 'month'])['total_ticket'].mean().round(2).reset_index()
panier_moyen_mois = panier_moyen_mois.rename(columns={'total_ticket': 'panier_moyen_mois'})

# Fusionner les données de panier moyen par semaine avec le DataFrame original
panier_ticket = panier_ticket.merge(panier_moyen_semaine, how='left', on=['year', 'week'])

# Fusionner les données de panier moyen par mois avec le DataFrame original
panier_ticket = panier_ticket.merge(panier_moyen_mois, how='left', on=['year', 'month'])

display(panier_ticket)

,date,year,month,week,time,ticket_number,total_ticket,panier_moyen_semaine,panier_moyen_mois
0,2021-01-02,2021,1,53,08:38:00,150040.0,4.50,4.97,3.89
1,2021-01-02,2021,1,53,09:14:00,150041.0,3.55,4.97,3.89
2,2021-01-02,2021,1,53,09:25:00,150042.0,6.00,4.97,3.89
3,2021-01-02,2021,1,53,09:25:00,150043.0,5.10,4.97,3.89
4,2021-01-02,2021,1,53,09:27:00,150044.0,1.05,4.97,3.89
...,...,...,...,...,...,...,...,...,...
136446,2022-09-30,2022,9,39,18:34:00,288908.0,2.70,3.60,3.94
136447,2022-09-30,2022,9,39,18:39:00,288910.0,1.30,3.60,3.94
136448,2022-09-30,2022,9,39,18:52:00,288911.0,12.15,3.60,3.94
136449,2022-09-30,2022,9,39,18:55:00,288912.0,1.30,3.60,3.94


### Téléchargement des datasets

In [ ]:
# Télécharger le dataset complet

# Format csv
# boulangerie.to_csv('boulangerie.csv', index = False, header=True)

# Format Excel
# boulangerie.to_excel('boulangerie.xlsx')

# Télécharger le dataset 'panier_total'
# panier_ticket.to_csv('panier_total.csv', index = False, header=True)